This notebook demonstrates integration of functionality from SNT with Python through pyimagej.  
The context is the computation and rendering of the convex hull enclosing all axonal/dendritic terminal nodes.    

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge pyimagej openjdk=8 scipy

In [ ]:
import imagej
import numpy as np
from scipy.spatial import ConvexHull

In [ ]:
# Initialize Fiji with GUI support.
ij = imagej.init('sc.fiji:fiji', headless=False)
from jnius import autoclass, cast

In [ ]:
# Import relevant SNT (Java) classes.
HashSet = autoclass('java.util.HashSet')
PointInImage = autoclass('tracing.util.PointInImage')
MouseLightLoader = autoclass('tracing.io.MouseLightLoader')
Tree = autoclass('tracing.Tree')
TreeAnalyzer = autoclass('tracing.analysis.TreeAnalyzer')
Color = autoclass('org.scijava.util.Colors')
Viewer = autoclass('tracing.viewer.Viewer3D')

In [ ]:
def convex_hull(tree):
        """computes the convex hull of the input tree"""
        
        analyzer = TreeAnalyzer(tree)
        # Extract the end points from the tree as a java HashSet.
        tips_java_set = analyzer.getTips()
        # Convert to array to maintain ordering.
        tips_java_array = tips_java_set.toArray()

        # Convert to python list.
        tips_list = [[t.x, t.y, t.z] for t in tips_java_array]

        assert len(tips_list) == tips_java_set.size()

        # Find the convex hull of the end points.
        hull = ConvexHull(tips_list)
        
        print("The volume of the convex hull encapsulating all tip points is {} cubic microns".format(
            round(hull.volume, 2)))
        
        print("The area of the hull is {} square microns".format(round(hull.area, 2)))
        print()
        
        # Find original PointInImage objects to pass to Viewer3D.
        # This is less costly than re-constructing a new Java object for each vertex.
        verts_java = [tips_java_array[i] for i in hull.vertices]
        verts_java = ij.py.to_java(verts_java)
        
        return verts_java

In [ ]:
def run():
    
    # Fetch swc from MouseLight database by ID.
    loader = MouseLightLoader('AA0100')
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somewhow the specified id was not found", "Error")
        return

    tree_axon = loader.getTree('axon', None)
    tree_dendrites = loader.getTree('dendrite', None)
    
        
    verts_axon = convex_hull(tree_axon)
    verts_dendrites = convex_hull(tree_dendrites)

    # Visualize the result using SNT Viewer3D.
    viewer = Viewer()
    viewer.add(tree_axon)
    viewer.add(tree_dendrites)
    viewer.addSurface(verts_axon, Color.RED, 50)
    viewer.addSurface(verts_dendrites, Color.CYAN, 50)
    viewer.show()

In [ ]:
run()